In [3]:
%pip install datasets transformers torch ipywidgets git-lfs

Note: you may need to restart the kernel to use updated packages.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("mideind/IceBERT")

model = AutoModelForMaskedLM.from_pretrained("mideind/IceBERT")

In [6]:
print(transformers.__version__)

4.26.0


In [7]:
squad_v2 = True
model_checkpoint = "mideind/IceBERT"
batch_size = 16

In [8]:
from datasets import load_dataset, load_metric

# data_files = {"train": "/work/GuðjónKristjánsson#6527/my_folder_gudjon/NLP-AU/Lokaverkefni/data/nqii_train_squad_format_tok.json"}

# datasets = load_dataset("json", data_files=data_files, field="data")

datasets = load_dataset("vesteinn/icelandic-qa-NQiI")



Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset icelandic-qa-n_qi_i downloaded and prepared to /home/coder/.cache/huggingface/datasets/vesteinn___icelandic-qa-n_qi_i/icelandic-qa-NQiI/1.0.0/76d8bc1fdbcf6396ea28a2f13e85a3e56dcc8eb54eb6386e7725409d2b854170. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4644
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 526
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 606
    })
})

In [10]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,nqii_3318,Jeremy Bentham,"Jeremy Bentham ( 15. febrúar 1748 – 6. júní 1832 ) var enskur lögfræðingur , heimspekingur umbótamaður og afkastamikill rithöfundur . Faðir hans og föðurfaðir voru lögfræðingar og lögmenn ( og gott ef ekki dómarar ) og hann fór þessa sömu leið í menntun sinni en starfaði þó lítið sem ekkert við það þar sem hann tók stóran arf ungur og vann nær ekkert á sinni æfi . Hann var róttækur hugsuður og frumkvöðull í réttarheimspeki í hinum enskumælandi heimi . Hann er þekktastur fyrir að vera málsvari nytjastefnu í siðfræði .",Hver var frumkvöðull sósíalisma ?,"{'text': [], 'answer_start': []}"
1,nqii_11523,Fyrri heimsstyrjöldin,"Þann 28. júní 1914 skaut Gavrilo Princip Franz Ferdinand , erkihertoga Austurríkis-Ungverjalands og erfingja krúnunnar , og eiginkonu hans Sophie Chotek til bana í Sarajevo . Princip var meðlimur í þjóðernissinnuðu samtökunum Ung Bosnía , sem höfðu það á stefnuskrá sinni að sameina alla Suður-Slava í einu ríki , sjálfstæðu og óháðu Austurríki-Ungverjalandi . Morðið í Sarajevo hratt af stað atburðarás sem stigmagnaðist og leiddi til stríðs . Morðið var tilefni stríðsins en raunverulegar orsakir þess voru aftur á móti margvíslegar og flóknar .",Hvers vegna var Franz Ferdinand myrtur ?,"{'text': [' Princip var meðlimur í þjóðernissinnuðu samtökunum Ung Bosnía ,'], 'answer_start': [174]}"
2,nqii_10508,Sæmundaredda,"Sæmundaredda er frægt íslenskt skinnhandrit frá um 1280 . Í því er að finna fornan kveðskap sem venja er að skipta í tvo flokka , goðakvæði og hetjukvæði . Hvað „ edda “ þýðir er óvíst , en það tengist skáldskaparfræði ( sjá Snorra-Edda ) . Brynjólfur Sveinsson gaf Friðriki 3 . Danakonungi handritið árið 1643 og var það geymt á Konunglega bókasafninu í Kaupmannahöfn . Konungsbókin var afhent Íslendingum 21. apríl 1971 ásamt Flateyjarbók og er nú geymd á Stofnun Árna Magnússonar .",Hvar eru íslensku fornritin geymd ?,"{'text': [], 'answer_start': []}"
3,nqii_6088,Þrælahald,"Þrælahald kallast það þegar menn eru meðhöndlaðir , lagalega eða félagslega , sem eign annarra manna . Fyrr á tímum var þrælahald oft löglegt og eignarréttur landeigenda og annarra efnamanna náði ekki einungis til landareigna og húseigna heldur einnig til vinnuafls . Slíkir þrælar áttu ekki rétt á launagreiðslum og voru að öllu leyti háðir ákvörðunum eigenda sinna en þeir gátu þá skipað þeim fyrir verkum og hlutverkum og selt þá eins og hverja aðra eign . Einstaklingur gæti orðið þræll við fæðingu , kaupi eða handtöku .",Hvað búa margir blökkumenn á Íslandi ?,"{'text': [], 'answer_start': []}"
4,nqii_16281,Emil Thoroddsen,"Emil Thoroddsen ( 16. júní 1898 – 7. júlí 1944 ) var tónskáld , píanóleikari , leikskáld , myndlistamaður , listgagnrýnandi og þýðandi .",Hver voru helstu verk Jóns Leifs ?,"{'text': [], 'answer_start': []}"
5,nqii_17150,Grísk goðafræði,"Samkvæmt grískum goðsögnum var í upphafi gapandi tóm , sem nefndist "" Kaos "" ... Upp úr því tómi spruttu m.a. "" Gaia "" , móðir jörð og "" Tartaros "" hinn myrki undirheimur . Gaia gat af sér "" Úranos "" , himininn , og áttu þau síðan ýmis afkvæmi , heldur tröllsleg , fyrst svonefnda "" kýklópa "" , sem höfðu eitt auga á enni , og síðan tólf "" títana "" . Þegar kýklóparnir uxu úr grasi , ógnuðu þeir veldi föður síns , en þá steypti Úranos þeim niður til undirheima . Þetta líkaði Gaiu illa og hvatti hún títana til uppreisnar gegn Úranosi . Tók yngsti þeirra , "" Krónos "" , þeirri áeggjan , steypti Úranosi föður sínum í undirheima og gerðist nýr drottnari heimsins .",Hvernig varð heimurinn til samkvæmt grískri goðafræði ?,"{'text': [' í upphafi gapandi tóm , sem nefndist "" Kaos "" ... Upp úr því tómi spruttu m.a. "" Gaia "" , móðir jörð og "" Tartaros ""'], 'answer_start': [30]}"
6,nqii_11961,Agnar Þórðarson,"Agnar Þórðarson ( 11. september 1917 - 12. ágúst 2006 ) var íslenskur rithöfundur , fæddur í Reykjavík . Hann var einn Kleppssystkina en foreldrar þeirra voru Ellen Sveinsson og Þórður Sveinsson yfirlæknir . Ag

In [12]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [14]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.


for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [15]:
len(tokenizer(example["question"], example["context"])["input_ids"])

514

In [16]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [17]:

tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [18]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 269]

In [19]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

<s>Hver er hæsta byggingin í Berlín?</s></s>Flestar þekktar byggingar Berlínar eru í borgarhlutanum Mitte, enda var hann miðborg Berlínar fyrir skiptingu, en einnig miðborg Austur-Berlínar fyrir sameiningu. Þekktasta bygging borgarinnar er Brandenborgarhliðið. Það er jafnframt orðið að þjóðartákni Þjóðverja eftir fall Berlínarmúrsins. Múrinn var aðeins í nokkra metra fjarlægð frá hliðinu, sem stóð austan við múrinn, þannig að það myndaði nokkurs konar botnlanga í miðborg Austur-Berlínar. Skammt norður af hliðinu stendur hið fræga ríkisþinghús ( " Reichstag " ). Það var reist sem þinghús á keisaratíma Prússlands og þar var Weimar-lýðveldið stofnað. Þegar bruninn í húsinu átti sér stað 1933, notaði Hitler sér hann sem átyllu til að auka völd sín og mynda þriðja ríkið. Þinghús þetta er næstvinsælasti ferðamannastaður Þýskalands, á eftir dómkirkjunni í Köln. Við Friedrichstrasse stóð varðstöðin Checkpoint Charlie. Hún var landamærastöð milli bandaríska og sovéska hernámssvæðanna á tíma kal

In [20]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 7), (8, 13), (14, 23), (24, 25), (26, 32), (33, 34), (0, 0), (0, 0), (0, 7), (8, 15), (16, 25), (26, 34), (35, 38), (39, 40), (41, 47), (47, 55), (56, 57), (57, 61), (62, 63), (64, 68), (69, 72), (73, 77), (78, 85), (86, 94), (95, 100), (101, 110), (111, 112), (113, 115), (116, 122), (123, 130), (131, 137), (137, 138), (138, 141), (141, 146), (147, 152), (153, 163), (164, 165), (166, 170), (170, 175), (176, 183), (184, 195), (196, 198), (199, 204), (204, 206), (206, 212), (212, 213), (213, 218), (219, 220), (221, 224), (225, 227), (228, 237), (238, 243), (244, 246), (247, 253), (253, 254), (254, 256), (256, 258), (259, 268), (269, 274), (275, 279), (280, 288), (288, 291), (291, 295), (296, 297), (298, 299), (299, 304), (305, 308), (309, 315), (316, 317), (318, 324), (325, 330), (331, 339), (340, 343), (344, 348), (348, 351), (352, 353), (354, 357), (358, 362), (363, 369), (370, 373), (374, 380), (381, 382), (383, 389), (390, 392), (393, 396), (397, 404), (405, 412)

In [21]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

Hver Hver


In [22]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [23]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

The answer is not in this feature.


In [24]:
# print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
# print(answers["text"][0])

In [25]:
pad_on_right = tokenizer.padding_side == "right"

In [26]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [27]:
features = prepare_train_features(datasets['train'][:5])

In [28]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [29]:

from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at mideind/IceBERT were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at mideind/IceBERT and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-

In [30]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [31]:
from transformers import default_data_collator

data_collator = default_data_collator

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/gudjonk93/IceBERT-finetuned-squad-10 into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [33]:
trainer.train()

/home/coder/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4687
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2930
  Number of trainable parameters = 123853058


Epoch,Training Loss,Validation Loss
1,No log,1.761428
2,1.950900,1.520794
3,1.950900,1.501118
4,0.952900,1.569377
5,0.952900,1.751601
6,0.664700,1.862866
7,0.433600,1.888145
8,0.433600,2.076820
9,0.335000,2.123781
10,0.335000,2.151051


***** Running Evaluation *****
  Num examples = 527
  Batch size = 16
Saving model checkpoint to IceBERT-finetuned-squad-10/checkpoint-500
Configuration saved in IceBERT-finetuned-squad-10/checkpoint-500/config.json
Model weights saved in IceBERT-finetuned-squad-10/checkpoint-500/pytorch_model.bin
tokenizer config file saved in IceBERT-finetuned-squad-10/checkpoint-500/tokenizer_config.json
Special tokens file saved in IceBERT-finetuned-squad-10/checkpoint-500/special_tokens_map.json
tokenizer config file saved in IceBERT-finetuned-squad-10/tokenizer_config.json
Special tokens file saved in IceBERT-finetuned-squad-10/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

***** Running Evaluation *****
  Num examples = 527
  Batch size = 16
***** Running Evaluation *****
  Num examples = 527
  Batch size = 16
Saving model checkpoint to IceBERT-finetuned-squad-10/checkpoint-1000
Configuration saved in IceBERT-finetuned-squad-10/checkpoint-1000/config.json
Model weights saved in IceBERT-finetuned-squad-10/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in IceBERT-finetuned-squad-10/checkpoint-1000/tokenizer_config.json
Special tokens file saved in IceBERT-finetuned-squad-10/checkpoint-1000/special_tokens_map.json


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer config file saved in IceBERT-finetuned-squad-10/tokenizer_config.json
Special tokens file saved in IceBERT-finetuned-squad-10/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

***** Running Evaluation *****
  Num examples = 527
  Batch size = 16
***** Running Evaluation *****
  Num examples = 527
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2930, training_loss=0.7798030371552035, metrics={'train_runtime': 10073.5686, 'train_samples_per_second': 4.653, 'train_steps_per_second': 0.291, 'total_flos': 9185234241162240.0, 'train_loss': 0.7798030371552035, 'epoch': 10.0})

In [34]:
trainer.save_model("test-squad-trained")

Saving model checkpoint to test-squad-trained-10e
Configuration saved in test-squad-trained-10e/config.json
Model weights saved in test-squad-trained-10e/pytorch_model.bin
tokenizer config file saved in test-squad-trained-10e/tokenizer_config.json
Special tokens file saved in test-squad-trained-10e/special_tokens_map.json
Saving model checkpoint to IceBERT-finetuned-squad-10
Configuration saved in IceBERT-finetuned-squad-10/config.json
Model weights saved in IceBERT-finetuned-squad-10/pytorch_model.bin
tokenizer config file saved in IceBERT-finetuned-squad-10/tokenizer_config.json
Special tokens file saved in IceBERT-finetuned-squad-10/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/473M [00:00<?, ?B/s]

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/gudjonk93/IceBERT-finetuned-squad-10
   77c4dd7..195aefb  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}, 'dataset': {'name': 'icelandic-qa-n_qi_i', 'type': 'icelandic-qa-n_qi_i', 'config': 'icelandic-qa-NQiI', 'split': 'validation', 'args': 'icelandic-qa-NQiI'}}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/gudjonk93/IceBERT-finetuned-squad-10
   195aefb..20950cd  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [36]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [37]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([180,  12, 199, 199, 201, 193, 183, 197,  31, 153,  11, 160,  25, 155,
           0,   0]),
 tensor([190, 195, 196, 215, 198, 197,   0, 201,  38, 156,  14, 168,  32, 168,
           0,   0]))

In [38]:
n_best_size = 20

In [39]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [40]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [41]:
validation_features = datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["test"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 610
  Batch size = 16


In [43]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [44]:
max_answer_length = 30

In [45]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["test"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[]

In [46]:
datasets["test"][0]["answers"]

{'text': ['Seinni heimsstyrjöldin', 'Seinni heimsstyrjöldin'],
 'answer_start': [0, 0]}

In [47]:
import collections

examples = datasets["test"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [48]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [49]:
final_predictions = postprocess_qa_predictions(datasets["test"], validation_features, raw_predictions.predictions)

Post-processing 606 example predictions split into 610 features.


  0%|          | 0/606 [00:00<?, ?it/s]

In [50]:

metric = load_metric("squad_v2" if squad_v2 else "squad")

/tmp/ipykernel_254/2255871158.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


In [51]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["test"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 57.42574257425743,
 'f1': 75.39481007574807,
 'total': 606,
 'HasAns_exact': 19.93127147766323,
 'HasAns_f1': 57.35139142922089,
 'HasAns_total': 291,
 'NoAns_exact': 92.06349206349206,
 'NoAns_f1': 92.06349206349206,
 'NoAns_total': 315,
 'best_exact': 57.42574257425743,
 'best_exact_thresh': 0.0,
 'best_f1': 75.39481007574805,
 'best_f1_thresh': 0.0}

In [52]:
trainer.push_to_hub()

Saving model checkpoint to IceBERT-finetuned-squad-10
Configuration saved in IceBERT-finetuned-squad-10/config.json
Model weights saved in IceBERT-finetuned-squad-10/pytorch_model.bin
tokenizer config file saved in IceBERT-finetuned-squad-10/tokenizer_config.json
Special tokens file saved in IceBERT-finetuned-squad-10/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}, 'dataset': {'name': 'icelandic-qa-n_qi_i', 'type': 'icelandic-qa-n_qi_i', 'config': 'icelandic-qa-NQiI', 'split': 'validation', 'args': 'icelandic-qa-NQiI'}}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
